# Microscope Defect Detection - WORKING VERSION

This notebook provides **multiple working methods** for detecting defects in microscope images.

**Features:**
- 3 different detection methods (try them all!)
- Easy-to-adjust parameters
- Clear visualizations
- Works on real microscope images

**How to use:**
1. Run all cells in order
2. Try each detection method
3. Adjust sensitivity sliders to tune detection

## 1. Import Libraries

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import os
from pathlib import Path

# Make plots look nice
plt.style.use('default')
%matplotlib inline

print("Libraries loaded successfully!")
print(f"OpenCV version: {cv2.__version__}")

## 2. Load Your Image

Update the path below to point to your image:

In [ ]:
# CHANGE THIS to your image path
image_path = 'images/Image-1.jpg'

# Load the image
img = cv2.imread(image_path)

if img is None:
    print(f"ERROR: Could not load image from '{image_path}'")
    print("Please check:")
    print("  1. The file exists")
    print("  2. The path is correct")
    print("  3. You have read permissions")
else:
    print(f"✓ Image loaded successfully!")
    print(f"  Size: {img.shape[1]} x {img.shape[0]} pixels")
    print(f"  Channels: {img.shape[2]}")
    
    # Display the image
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(12, 8))
    plt.imshow(img_rgb)
    plt.title('Original Microscope Image', fontsize=16, fontweight='bold')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

## 3. METHOD 1: Adaptive Threshold Detection

**Best for:** Small defects, spots, particles

**How it works:** Compares each pixel to its local neighborhood

In [ ]:
def detect_defects_adaptive(img, sensitivity=15, min_area=10, max_area=1000):
    """
    Detect defects using adaptive thresholding
    
    Parameters:
    -----------
    img : image to process
    sensitivity : int (1-50)
        Higher = finds more defects but more false positives
        Lower = finds only obvious defects
        Recommended: 10-20
    min_area : int
        Minimum defect size in pixels
    max_area : int
        Maximum defect size in pixels
    """
    
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Apply slight blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Adaptive threshold - this is the key!
    # It compares each pixel to its local neighborhood
    thresh = cv2.adaptiveThreshold(
        blurred,
        255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV,
        blockSize=21,  # Size of neighborhood
        C=sensitivity   # Sensitivity adjustment
    )
    
    # Clean up the mask
    kernel = np.ones((3, 3), np.uint8)
    cleaned = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)
    cleaned = cv2.morphologyEx(cleaned, cv2.MORPH_CLOSE, kernel, iterations=1)
    
    # Find contours (defect boundaries)
    contours, _ = cv2.findContours(cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Filter by area
    valid_contours = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if min_area < area < max_area:
            valid_contours.append(cnt)
    
    return valid_contours, cleaned, gray


# Test it!
print("Running adaptive detection...")
contours, mask, gray = detect_defects_adaptive(img, sensitivity=15)

print(f"\n✓ Found {len(contours)} defects!\n")

# Visualize results
img_result = img.copy()
cv2.drawContours(img_result, contours, -1, (0, 255, 0), 2)

# Draw bounding boxes
for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
    cv2.rectangle(img_result, (x, y), (x+w, y+h), (255, 0, 0), 1)

# Display
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

axes[0, 0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('Original Image', fontsize=14, fontweight='bold')
axes[0, 0].axis('off')

axes[0, 1].imshow(gray, cmap='gray')
axes[0, 1].set_title('Grayscale', fontsize=14, fontweight='bold')
axes[0, 1].axis('off')

axes[1, 0].imshow(mask, cmap='gray')
axes[1, 0].set_title('Defect Mask (white = defect)', fontsize=14, fontweight='bold')
axes[1, 0].axis('off')

axes[1, 1].imshow(cv2.cvtColor(img_result, cv2.COLOR_BGR2RGB))
axes[1, 1].set_title(f'Detected Defects: {len(contours)}', fontsize=14, fontweight='bold', color='green')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

# Print details
print("\nDefect Details:")
print("-" * 60)
print(f"{'#':<5} {'Position (x,y)':<20} {'Size':<15} {'Area (px²)':<10}")
print("-" * 60)

for i, cnt in enumerate(contours[:20], 1):  # Show first 20
    area = cv2.contourArea(cnt)
    x, y, w, h = cv2.boundingRect(cnt)
    print(f"{i:<5} ({x:4d}, {y:4d})        {w:3d}×{h:<3d}        {area:7.1f}")

if len(contours) > 20:
    print(f"... and {len(contours)-20} more defects")

### Adjust Sensitivity

Try different sensitivity values:

In [ ]:
# Try different sensitivity values
sensitivities = [5, 10, 15, 20, 25]

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

# Show original
axes[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
axes[0].set_title('Original', fontsize=12, fontweight='bold')
axes[0].axis('off')

# Try each sensitivity
for i, sens in enumerate(sensitivities, 1):
    contours, _, _ = detect_defects_adaptive(img, sensitivity=sens)
    
    img_result = img.copy()
    cv2.drawContours(img_result, contours, -1, (0, 255, 0), 2)
    
    axes[i].imshow(cv2.cvtColor(img_result, cv2.COLOR_BGR2RGB))
    axes[i].set_title(f'Sensitivity={sens}\n{len(contours)} defects', fontsize=11)
    axes[i].axis('off')

plt.tight_layout()
plt.show()

print("\nRecommendation:")
print("  • Too many false positives? → Decrease sensitivity (try 8-12)")
print("  • Missing defects? → Increase sensitivity (try 18-25)")

## 4. METHOD 2: Background Subtraction

**Best for:** Detecting anything different from the background

**How it works:** Removes the estimated background, leaving only anomalies

In [ ]:
def detect_defects_background_subtraction(img, blur_size=51, threshold=15, min_area=10, max_area=1000):
    """
    Detect defects by subtracting estimated background
    
    Parameters:
    -----------
    img : image to process
    blur_size : int (must be odd, 31-101)
        Size of background estimation blur
        Larger = smoother background, detects smaller defects
    threshold : int (1-50)
        Sensitivity threshold
        Lower = more sensitive, finds more defects
    """
    
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Ensure blur_size is odd
    if blur_size % 2 == 0:
        blur_size += 1
    
    # Estimate background by heavy blurring
    background = cv2.GaussianBlur(gray, (blur_size, blur_size), 0)
    
    # Subtract background (this isolates defects!)
    diff = cv2.absdiff(gray, background)
    
    # Apply threshold to get binary mask
    _, thresh = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)
    
    # Clean up
    kernel = np.ones((3, 3), np.uint8)
    cleaned = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)
    cleaned = cv2.morphologyEx(cleaned, cv2.MORPH_CLOSE, kernel, iterations=2)
    
    # Find contours
    contours, _ = cv2.findContours(cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Filter by area
    valid_contours = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if min_area < area < max_area:
            valid_contours.append(cnt)
    
    return valid_contours, diff, background


# Test it!
print("Running background subtraction detection...")
contours, diff, background = detect_defects_background_subtraction(img, blur_size=51, threshold=15)

print(f"\n✓ Found {len(contours)} defects!\n")

# Visualize
img_result = img.copy()
cv2.drawContours(img_result, contours, -1, (0, 255, 0), 2)

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

axes[0, 0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('Original Image', fontsize=14, fontweight='bold')
axes[0, 0].axis('off')

axes[0, 1].imshow(background, cmap='gray')
axes[0, 1].set_title('Estimated Background', fontsize=14, fontweight='bold')
axes[0, 1].axis('off')

axes[1, 0].imshow(diff, cmap='hot')
axes[1, 0].set_title('Difference (defects highlighted)', fontsize=14, fontweight='bold')
axes[1, 0].axis('off')

axes[1, 1].imshow(cv2.cvtColor(img_result, cv2.COLOR_BGR2RGB))
axes[1, 1].set_title(f'Detected Defects: {len(contours)}', fontsize=14, fontweight='bold', color='green')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

## 5. METHOD 3: Edge/Scratch Detection

**Best for:** Lines, scratches, cracks

**How it works:** Detects strong edges in the image

In [ ]:
def detect_defects_edges(img, low_threshold=30, high_threshold=100, min_length=20):
    """
    Detect line defects (scratches, cracks) using edge detection
    
    Parameters:
    -----------
    img : image to process
    low_threshold : int (10-100)
        Lower edge detection threshold
    high_threshold : int (50-200)
        Upper edge detection threshold
    min_length : int
        Minimum line length to consider as defect
    """
    
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Apply blur
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Canny edge detection
    edges = cv2.Canny(blurred, low_threshold, high_threshold)
    
    # Dilate edges to connect nearby edges
    kernel = np.ones((3, 3), np.uint8)
    dilated = cv2.dilate(edges, kernel, iterations=2)
    
    # Find contours of edge regions
    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Filter by perimeter (length)
    valid_contours = []
    for cnt in contours:
        perimeter = cv2.arcLength(cnt, True)
        if perimeter > min_length:
            valid_contours.append(cnt)
    
    # Detect lines using Hough transform
    lines = cv2.HoughLinesP(
        edges,
        rho=1,
        theta=np.pi/180,
        threshold=50,
        minLineLength=min_length,
        maxLineGap=10
    )
    
    return valid_contours, edges, lines


# Test it!
print("Running edge detection...")
contours, edges, lines = detect_defects_edges(img, low_threshold=30, high_threshold=100)

print(f"\n✓ Found {len(contours)} edge defects!")
if lines is not None:
    print(f"✓ Found {len(lines)} line defects!\n")
else:
    print("✓ Found 0 line defects\n")

# Visualize
img_result = img.copy()

# Draw contours
cv2.drawContours(img_result, contours, -1, (0, 255, 0), 2)

# Draw lines
if lines is not None:
    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(img_result, (x1, y1), (x2, y2), (255, 0, 0), 2)

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

axes[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
axes[0].set_title('Original Image', fontsize=14, fontweight='bold')
axes[0].axis('off')

axes[1].imshow(edges, cmap='gray')
axes[1].set_title('Detected Edges', fontsize=14, fontweight='bold')
axes[1].axis('off')

axes[2].imshow(cv2.cvtColor(img_result, cv2.COLOR_BGR2RGB))
line_count = len(lines) if lines is not None else 0
axes[2].set_title(f'Detected: {len(contours)} regions, {line_count} lines', fontsize=14, fontweight='bold', color='green')
axes[2].axis('off')

plt.tight_layout()
plt.show()

## 6. COMBINED DETECTION

Use all methods together for best results!

In [ ]:
def detect_all_defects(img):
    """
    Combine all detection methods
    """
    
    print("Running combined detection...\n")
    
    # Method 1: Adaptive
    print("  [1/3] Adaptive threshold detection...")
    adaptive_contours, _, _ = detect_defects_adaptive(img, sensitivity=15)
    print(f"        → Found {len(adaptive_contours)} defects")
    
    # Method 2: Background subtraction
    print("  [2/3] Background subtraction...")
    bg_contours, _, _ = detect_defects_background_subtraction(img, blur_size=51, threshold=15)
    print(f"        → Found {len(bg_contours)} defects")
    
    # Method 3: Edge detection
    print("  [3/3] Edge detection...")
    edge_contours, _, _ = detect_defects_edges(img)
    print(f"        → Found {len(edge_contours)} defects")
    
    # Visualize all results
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Original
    axes[0, 0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    axes[0, 0].set_title('Original Image', fontsize=14, fontweight='bold')
    axes[0, 0].axis('off')
    
    # Method 1
    img1 = img.copy()
    cv2.drawContours(img1, adaptive_contours, -1, (0, 255, 0), 2)
    axes[0, 1].imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
    axes[0, 1].set_title(f'Method 1: Adaptive\n{len(adaptive_contours)} defects', fontsize=13, fontweight='bold')
    axes[0, 1].axis('off')
    
    # Method 2
    img2 = img.copy()
    cv2.drawContours(img2, bg_contours, -1, (255, 0, 0), 2)
    axes[1, 0].imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
    axes[1, 0].set_title(f'Method 2: Background Subtraction\n{len(bg_contours)} defects', fontsize=13, fontweight='bold')
    axes[1, 0].axis('off')
    
    # Method 3
    img3 = img.copy()
    cv2.drawContours(img3, edge_contours, -1, (0, 0, 255), 2)
    axes[1, 1].imshow(cv2.cvtColor(img3, cv2.COLOR_BGR2RGB))
    axes[1, 1].set_title(f'Method 3: Edge Detection\n{len(edge_contours)} defects', fontsize=13, fontweight='bold')
    axes[1, 1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print("\n" + "="*60)
    print("SUMMARY")
    print("="*60)
    print(f"Method 1 (Adaptive):              {len(adaptive_contours):5d} defects")
    print(f"Method 2 (Background Subtraction): {len(bg_contours):5d} defects")
    print(f"Method 3 (Edge Detection):         {len(edge_contours):5d} defects")
    print("="*60)
    
    return adaptive_contours, bg_contours, edge_contours


# Run it!
adaptive, background, edges = detect_all_defects(img)

## 7. Process Multiple Images

Process all images in a folder:

In [ ]:
def process_folder(folder_path, method='adaptive', sensitivity=15):
    """
    Process all images in a folder
    
    Parameters:
    -----------
    folder_path : str
        Path to folder containing images
    method : str
        Detection method: 'adaptive', 'background', or 'edges'
    sensitivity : int
        Sensitivity parameter (meaning depends on method)
    """
    
    # Find all images
    image_extensions = ['.jpg', '.jpeg', '.png', '.tiff', '.tif', '.bmp']
    image_files = []
    
    for filename in os.listdir(folder_path):
        if any(filename.lower().endswith(ext) for ext in image_extensions):
            image_files.append(filename)
    
    if not image_files:
        print(f"No images found in '{folder_path}'")
        return
    
    print(f"Found {len(image_files)} images")
    print(f"Using method: {method}")
    print(f"Sensitivity: {sensitivity}\n")
    print("="*70)
    
    results = []
    
    for i, filename in enumerate(image_files[:10], 1):  # Process first 10
        print(f"[{i}/{min(10, len(image_files))}] {filename}")
        
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path)
        
        if img is None:
            print("     ERROR: Could not load\n")
            continue
        
        # Detect based on method
        if method == 'adaptive':
            contours, _, _ = detect_defects_adaptive(img, sensitivity=sensitivity)
        elif method == 'background':
            contours, _, _ = detect_defects_background_subtraction(img, threshold=sensitivity)
        else:
            contours, _, _ = detect_defects_edges(img, low_threshold=sensitivity)
        
        print(f"     → {len(contours)} defects\n")
        
        results.append({
            'filename': filename,
            'defect_count': len(contours)
        })
    
    # Summary
    print("="*70)
    print("BATCH SUMMARY")
    print("="*70)
    total_defects = sum(r['defect_count'] for r in results)
    print(f"Total images processed: {len(results)}")
    print(f"Total defects found: {total_defects}")
    print(f"Average defects per image: {total_defects/len(results):.1f}")
    print("\nPer-image results:")
    for r in results:
        print(f"  {r['filename']:<40} {r['defect_count']:>5} defects")
    
    return results


# Example: Process images folder
results = process_folder('images/', method='adaptive', sensitivity=15)

## 8. Save Results

Save annotated images:

In [ ]:
def save_annotated_image(img_path, output_path, method='adaptive', sensitivity=15):
    """
    Save an image with defects highlighted
    """
    
    # Load image
    img = cv2.imread(img_path)
    if img is None:
        print(f"Error loading {img_path}")
        return
    
    # Detect defects
    if method == 'adaptive':
        contours, _, _ = detect_defects_adaptive(img, sensitivity=sensitivity)
    elif method == 'background':
        contours, _, _ = detect_defects_background_subtraction(img, threshold=sensitivity)
    else:
        contours, _, _ = detect_defects_edges(img)
    
    # Draw detections
    img_result = img.copy()
    cv2.drawContours(img_result, contours, -1, (0, 255, 0), 2)
    
    # Add text
    text = f"Defects: {len(contours)}"
    cv2.putText(img_result, text, (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # Save
    cv2.imwrite(output_path, img_result)
    print(f"✓ Saved: {output_path} ({len(contours)} defects)")


# Example: Save result for Image-1
save_annotated_image(
    'images/Image-1.jpg',
    'result_Image-1.jpg',
    method='adaptive',
    sensitivity=15
)

## 9. Quick Reference

### Which method should I use?

| Defect Type | Best Method | Key Parameter |
|-------------|-------------|---------------|
| Small spots, particles | Adaptive | sensitivity=10-20 |
| Texture variations | Background Subtraction | threshold=10-20 |
| Lines, scratches | Edge Detection | low_threshold=20-50 |
| All of the above | Use all 3 methods! | - |

### Parameter Tuning Guide

**If you see too many false positives:**
- Adaptive: Decrease `sensitivity` (try 8-12)
- Background: Increase `threshold` (try 20-30)
- Edges: Increase `low_threshold` (try 40-60)

**If missing real defects:**
- Adaptive: Increase `sensitivity` (try 18-25)
- Background: Decrease `threshold` (try 5-10)
- Edges: Decrease `low_threshold` (try 15-25)

### Common Issues

**Problem: Detecting nothing**
- Check image loaded correctly
- Increase sensitivity parameters
- Reduce `min_area` requirement

**Problem: Everything is a defect**
- Decrease sensitivity
- Increase `min_area` to filter noise
- Try background subtraction with larger blur_size

**Problem: Only detecting edges**
- Don't use edge detection method
- Use adaptive or background subtraction instead

## 10. Test on Your Own Image

Quick test cell - just change the path and run!

In [ ]:
# ===== QUICK TEST =====
# Just change these values and run!

TEST_IMAGE = 'images/Image-1.jpg'  # ← Change this
METHOD = 'adaptive'                 # ← 'adaptive', 'background', or 'edges'
SENSITIVITY = 15                    # ← Adjust this (10-25 usually works)

# Load and process
test_img = cv2.imread(TEST_IMAGE)

if test_img is None:
    print(f"ERROR: Could not load '{TEST_IMAGE}'")
else:
    print(f"Processing: {TEST_IMAGE}")
    print(f"Method: {METHOD}")
    print(f"Sensitivity: {SENSITIVITY}\n")
    
    # Detect
    if METHOD == 'adaptive':
        contours, mask, gray = detect_defects_adaptive(test_img, sensitivity=SENSITIVITY)
    elif METHOD == 'background':
        contours, mask, gray = detect_defects_background_subtraction(test_img, threshold=SENSITIVITY)
    else:
        contours, mask, gray = detect_defects_edges(test_img, low_threshold=SENSITIVITY)
    
    print(f"✓ Found {len(contours)} defects!\n")
    
    # Show result
    result = test_img.copy()
    cv2.drawContours(result, contours, -1, (0, 255, 0), 2)
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    axes[0].imshow(cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB))
    axes[0].set_title('Original', fontsize=14, fontweight='bold')
    axes[0].axis('off')
    
    axes[1].imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
    axes[1].set_title(f'Detected: {len(contours)} defects', fontsize=14, fontweight='bold', color='green')
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()